<a href="https://colab.research.google.com/github/Summit2/ML_education/blob/main/Comparison_of_GD_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

import seaborn as sns
sns.set(style="darkgrid", font_scale=1.4)
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.nn import functional as F

from torch.utils.data import TensorDataset, DataLoader

from torchvision.datasets import MNIST
from torchvision import transforms


In [3]:
data_transforms = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))

    ]

)

In [4]:
root = './'
train_dataset = MNIST(root, transform=data_transforms, download = True)
val_dataset = MNIST(root, transform=data_transforms,download = True)

train_dataloader = DataLoader(train_dataset, batch_size = 128)
val_dataloader =  DataLoader(val_dataset, batch_size = 128)

loaders = {'train' : train_dataloader,
           'valid' : val_dataloader }

100%|██████████| 9912422/9912422 [00:00<00:00, 59235798.25it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 70427728.97it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 20617126.54it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10738742.26it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [5]:

LeNet = nn.Sequential(
    nn.Conv2d(1,6,3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(6,16,3),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(400,120),
    nn.ReLU(),
    nn.Linear(120,84),
    nn.ReLU(),
    nn.Linear(84,10)

)


In [6]:
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device

'cpu'

In [1]:
def train_loop(optimizer):
  model = LeNet()
  model = model.to(device)

  criterion = torch.nn.CrossEntropyLoss()
  optimizer = optimizer

  max_epochs = 10
  accuracy = {"train": [], "valid": []}



  for epoch in range(max_epochs):
    for k, dataloader in loaders.items():
      for x_batch, y_batch in dataloader:
        x_batch, y_batch = x_batch.to(device),y_batch.to(device)

        if k == "train":
          model.train()
          optimizer.zero_grad()
          outp = model(x_batch)
          loss = criterion(outp, y_batch)
          loss.backward()
          optimizer.step()


        elif k == "valid":
          model.eval()
          with torch.no_grad():
            outp = model(x_batch)

        #посчитаем accuracy

        all_answers = len(outp)
        correct_answers = 1
